In [31]:
# 최종 데이터 읽어오기 (변수 5개 (Np, Vp, Bz, Bt, Bl)만 남긴 combine)

import numpy as np
import pandas as pd

np.random.seed(5)

path='C:\Projects\keras_talk\comp_model_data\\'

combine_data = pd.read_csv(path + 'combine_interpolation_an.csv', engine='python') 

kp_data = pd.read_csv(path + 'kp_data.csv') # 지자기교란 지수 파일에서 index부분 없애고 kp_data로 이름 바꿈

In [32]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

In [38]:
def create_dataset(signal_data, result_data, look_back):
    dataX, dataY = [], []
    for i in range(0, len(signal_data)-12, 12):
        dataX.append(signal_data[i:(i+look_back)]) # look_back = 24 단위로 6시간씩 끊어짐 - [0:24],[24:48],...
    for j in range(0, len(result_data)):
        for k in range(0, 8):
            dataY.append(result_data[j][k]) # kp지수 데이터의 하나 하나를 뽑아내어 6시간의 combine 데이터와 대응시킴
    return np.array(dataX), np.array(dataY)

look_back = 24

In [39]:
# 1. 데이터셋 생성하기

signal_data = combine_data.values
result_data = kp_data.values

x_train = signal_data[0:385728] # 2920*12*8 + 2928*12*3 (00,04,08년 윤년) = 385728 (99~09년까지의 데이터를 train으로)
x_val = signal_data[385728:455808] # 2920*12*2=70080 (10~11년까지의 데이터를 val으로)
x_test = signal_data[455808:] # 2920*12*1 + 2928*12*1 =70080 (12년 윤년) (12~13년까지의 데이터를 test으로)

y_train = result_data[0:4018] # 99~09년도
y_val = result_data[4018:4748] # 10~11년도
y_test = result_data[4748:] # 12~13년도

# 데이터셋 생성

x_train, y_train = create_dataset(x_train, y_train, look_back)
x_val, y_val = create_dataset(x_val, y_val, look_back)
x_test, y_test = create_dataset(x_test, y_test, look_back)

y_train = y_train[1:]
y_val = y_val[1:]
y_test = y_test[1:]

In [40]:
# 데이터셋 전처리 (RNN은 3차원으로 처리를 해야 한다고 함)
# 변수 5개 (Np, Vp, Bz, Bt, Bl)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 5) # (number, timestep, feature)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 5)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 5)

In [41]:
# 원핫 인코딩 (어떤 값을 리스트 내에 0,1로 표시하는 벡터로 표현하도록 함, 예를 들어 3인 경우엔 [0,0,0,1,0,0,0,0,0,0]으로 표현됨)

from keras.utils import to_categorical

size = 10

y_train = to_categorical(y_train, num_classes=size)
y_val = to_categorical(y_val, num_classes=size)
y_test = to_categorical(y_test, num_classes=size)

In [42]:
# 2. 모델 구성하기

from keras.optimizers import Adam

Optimizer = Adam(lr = 0.001)

model = Sequential()
model.add(LSTM(48,input_shape=(24,5)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 3. 모델 학습과정 설정하기

model.compile(loss='mean_squared_error', optimizer=Optimizer, metrics=['accuracy'])

# 4. 모델 학습시키기
hist = model.fit(x_train, y_train, epochs=100, batch_size=50, validation_data=(x_val, y_val))

ValueError: Input arrays should have the same number of samples as target arrays. Found 16072 input samples and 32143 target samples.

In [ ]:
# 5. 모델 평가하기

trainScore = model.evaluate(x_train, y_train, verbose=0)
model.reset_states()
print('Train Score: ', trainScore)
valScore = model.evaluate(x_val, y_val, verbose=0)
model.reset_states()
print('Validataion Score: ', valScore)
testScore = model.evaluate(x_test, y_test, verbose=0)
model.reset_states()
print('Test Score: ', testScore)

In [15]:
# 6. 모델 사용하기

path='C:\Projects\keras_talk\comp_model_data\\'

# 룩백 12일때는 학습 데이터가 한 셀에 3시간치만 있게 되는데 
# 룩백 24일때는 학습 데이터가 한 셀에 6시간치가 있어서 그것에 맞게 문제 데이터를 create_dataset으로 한 번 더 처리
# 반대로 룩백 12일때는 그게 기본 설정이라 문제 데이터를 굳이 처리할 필요가 없었던 것

def create_dataset(signal_data,look_back):
    dataX = []
    for i in range(0, len(signal_data)-12, 12): # look_back = 24 단위로 6시간씩 끊어짐
        dataX.append(signal_data[i:(i+look_back)]) # [0:24],[24:48],..
    return np.array(dataX)

look_back = 24

# 룩백이 24, 즉 6시간 뒤부터 보는 거라서 원래 problem_interpolation1 파일의 맨 앞에 전년도 마지막 3시간 데이터를 엑셀로 직접 덧붙임
# 그렇게 problem_interpolation2를 만듦

xhat = pd.read_csv(path + 'problem_interpolation2.csv', engine='python')

input_data = xhat.values

input_data = create_dataset(input_data, look_back)

#input_data = input_data.reshape(input_data.shape[0], input_data.shape[1], 1)

#input_data = input_data.reshape(1460, 12, 5) - 이 두 줄이 필요한지 안한지는 실행해봐야 알 것 같음.

forecast_result = model.predict(input_data)

In [17]:
# 예측한 kp지수를 엑셀로 다시 정리

result = []

for i in range(0,2920):
    result.append(np.argmax(forecast_result[i]))

final_result = []

for i in range(0, len(result), 8):
    final_result.append(result[i:i+8])

kp_0h = []
kp_3h = []
kp_6h = []
kp_9h = []
kp_12h = []
kp_15h = []
kp_18h = []
kp_21h = []

for i in range(0, len(final_result)):
    kp_0h.append(final_result[i][0])
    kp_3h.append(final_result[i][1])
    kp_6h.append(final_result[i][2])
    kp_9h.append(final_result[i][3])
    kp_12h.append(final_result[i][4])
    kp_15h.append(final_result[i][5])
    kp_18h.append(final_result[i][6])
    kp_21h.append(final_result[i][7])

from collections import OrderedDict

raw_data = OrderedDict()
raw_data['kp_0h'] = kp_0h
raw_data['kp_3h'] = kp_3h
raw_data['kp_6h'] = kp_6h
raw_data['kp_9h'] = kp_9h
raw_data['kp_12h'] = kp_12h
raw_data['kp_15h'] = kp_15h
raw_data['kp_18h'] = kp_18h
raw_data['kp_21h'] = kp_21h

kp_predict = pd.DataFrame(raw_data)

kp_predict.to_csv('./kp_predict.csv', index=False)